In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
import math
import pyparsing as pp

In [ ]:
data = load_data(2023, 19)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}""",
        19114,
        167409079868000,
    ),
]

# Part 1

In [ ]:
def parse_workflows(data):
    uint = pp.Word(pp.nums).set_parse_action(lambda toks: int(toks[0]))
    label = pp.Word(pp.alphas)
    category = pp.Or(pp.Keyword(c) for c in "xmas")
    comparator = (
        pp.Word(">").set_parse_action(lambda: int.__gt__)
        | pp.Word("<").set_parse_action(lambda: int.__lt__)
    )
    step = pp.Group(category + comparator + uint + pp.Suppress(":") + label) | label
    workflow = pp.Group(
        label
        + pp.Suppress("{")
        + pp.Group(pp.delimited_list(step))
        + pp.Suppress("}")
    )
    part = pp.Dict(
        pp.Suppress("{")
        + pp.delimited_list(pp.Group(category + pp.Suppress("=") + uint))
        + pp.Suppress("}"),
        asdict=True,
    )
    block = pp.Dict(pp.OneOrMore(workflow), asdict=True) + pp.Group(pp.OneOrMore(part), aslist=True)

    workflows, parts = block.parse_string(data)
    workflows["A"] = ["A"]
    workflows["R"] = ["R"]
    return workflows, parts

In [ ]:
def outcome(part, workflows):
    label, step = "in", 0
    while True:
        match workflows[label][step]:
            case "A":
                return sum(part.values())
            case "R":
                return 0
            case category, cmp, int(threshold), next:
                if cmp(part[category], threshold):
                    label = next
                    step = 0
                else:
                    step += 1
            case str(next):
                label = next
                step = 0
            case _:
                raise ValueError(f"Invalid workflow: {label, step}")

In [ ]:
def accepted(data):
    workflows, parts = parse_workflows(data)
    return sum(outcome(part, workflows) for part in parts)

In [ ]:
check(accepted, tests)
accepted(data)

# Part 2

In [ ]:
def accepted_count(workflows):
    ranges = [({c: (1, 4000) for c in "xmas"}, "in", 0)]
    total = 0
    while ranges:
        rng, label, step = ranges.pop()
        match workflows[label][step]:
            case "A":
                total += math.prod(max_ - min_ + 1 for min_, max_ in rng.values())
            case "R":
                pass
            case category, cmp, int(threshold), next:
                left = (next, 0)
                right = (label, step + 1)
                if cmp == int.__gt__:
                    cmp = int.__lt__
                    threshold += 1
                    left, right = right, left
                min_, max_ = rng[category]
                if cmp(min_, threshold):
                    left_range = rng.copy()
                    left_range[category] = min_, min(max_, threshold - 1)
                    ranges.append((left_range, *left))
                if not cmp(max_, threshold):
                    right_ranges = rng.copy()
                    right_ranges[category] = max(min_, threshold), max_
                    ranges.append((right_ranges, *right))
            case str(next):
                ranges.append((rng, next, 0))
            case _:
                raise ValueError(f"Invalid workflow: {label, step}")
    return total

In [ ]:
def combinations(data):
    workflows, _ = parse_workflows(data)
    return accepted_count(workflows)

In [ ]:
check(combinations, tests, 2)
combinations(data)